<a href="https://colab.research.google.com/github/Sahil2927/SASOLTRS/blob/main/Comparison_Bewtween_Sentiment_Analysis_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers tensorflow datasets vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from google.colab import files

In [ ]:
uploaded = files.upload()
df = pd.read_csv(next(iter(uploaded)))
df = df[['Title', 'Rating', 'Review']].dropna()
df['label'] = df['Rating'].apply(lambda x: 1 if x >= 7 else 0)
df = df.dropna(subset=['Review', 'label'])
texts = list(df['Review'].astype(str))
labels = df['label'].values

X_train, X_test, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)

Saving cleaned_dataset.csv to cleaned_dataset.csv


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from google.colab import files

transformer_models = {
    "BERT": "bert-base-uncased",
    "DistilBERT": "distilbert-base-uncased",
    "RoBERTa": "roberta-base",
    "ELECTRA": "google/electra-base-discriminator",
    "MiniLM": "nreimers/MiniLM-L6-H384-uncased",
    "DeBERTa": "microsoft/deberta-base",
    "ALBERT": "albert-base-v2",
}

def train_transformer(model_name, model_path, X_train, X_test, y_train, y_test):
    print(f"\n🔧 Fine-tuning {model_name}...")

    tokenizer = AutoTokenizer.from_pretrained(model_path)
    train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=128, return_tensors='tf')
    test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=128, return_tensors='tf')

    train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train)).batch(16)
    test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), y_test)).batch(16)

    try:
        # Try loading normally first
        model = TFAutoModelForSequenceClassification.from_pretrained(
            model_path,
            num_labels=2
        )
    except Exception as e:
        print(f"⚠️ {model_name} has no native TF weights. Loading PyTorch weights instead...")
        try:
            model = TFAutoModelForSequenceClassification.from_pretrained(
                model_path,
                num_labels=2,
                from_pt=True   # <-- Load PyTorch weights into TF
            )
        except Exception as e2:
            print(f"❌ Failed to load {model_name}: {e2}")
            return None

    # Compile & Train
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )

    model.fit(train_dataset, validation_data=test_dataset, epochs=2)

    # Evaluate
    loss, accuracy = model.evaluate(test_dataset)
    print(f"✔️ {model_name} Accuracy: {accuracy:.4f}")

    y_pred_logits = model.predict(test_dataset).logits
    y_pred = np.argmax(y_pred_logits, axis=1)
    y_true = np.concatenate([y for _, y in test_dataset], axis=0)

    print(f"\n📊 Classification Report for {model_name}:\n")
    print(classification_report(y_true, y_pred))

    return accuracy



In [ ]:
results = {}

for name, path in transformer_models.items():
    print(f"\n🔍 Loading model: {name}")
    acc = train_transformer(name, path, X_train, X_test, y_train, y_test)
    if acc is not None: # Only store accuracy if model loading and training was successful
        results[name] = acc


print("\n===========================")
print("📈 Final Accuracy Summary")
print("===========================")
for model, acc in results.items():
    print(f"{model:<12}: {acc:.4f}")


🔍 Loading model: BERT

🔧 Fine-tuning BERT...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

⚠️ ALBERT has no native TF weights. Loading PyTorch weights instead...


pytorch_model.bin:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFAlbertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFAlbertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2
475/475 [==============================] - 236s 460ms/step - loss: 0.6491 - accuracy: 0.6543 - val_loss: 0.6343 - val_accuracy: 0.6742
Epoch 2/2
119/119 [==============================] - 18s 155ms/step - loss: 0.6332 - accuracy: 0.6742
✔️ ALBERT Accuracy: 0.6742
119/119 [==============================] - 22s 158ms/step

📊 Classification Report for ALBERT:

              precision    recall  f1-score   support

           0       0.67      1.00      0.81      1279
           1       0.00      0.00      0.00       618

    accuracy                           0.67      1897
   macro avg       0.34      0.50      0.40      1897
weighted avg       0.45      0.67      0.54      1897


📈 Final Accuracy Summary
BERT        : 0.7343
DistilBERT  : 0.7428
RoBERTa     : 0.6742
ELECTRA     : 0.6916
MiniLM      : 0.7354
DeBERTa     : 0.6742
ALBERT      : 0.6742


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
